# Projeto final

Bem vindo ao projeto final do curso/módulo de Pandas.
Neste projeto, você deve responder a um conjunto de perguntas feitas a dois conjuntos de dados:

1. [Obesity among adults by country, 1975-2016](https://www.kaggle.com/amanarora/obesity-among-adults-by-country-19752016/)
2. [GDP Per Person (1901-2011)](https://www.kaggle.com/divyansh22/gdp-per-person-19012011?select=GDP.csv)

Tais conjuntos de dados são públicos foram publicados no Kaggle. Mais instruções sobre cada um deles podem ser encontrados nos links acima. Você também os encontrará na pasta `Datasets`, dentro de nossas apostilas.

In [26]:
from IPython.display import display
import pandas as pd
import numpy as np
import sqlite3 as sql

aux = pd.DataFrame()

def get_compact_df(path, dropcols=[], clearna=['any', 'all']):
    """ Função para importar um DataFrame de um arquivo csv, permite tratamento das DFs
    :param path:        Caminho do qual se deve importar
    :param dropcols:    Lista de colunas a remover, caso não sejam úteis
    :param clearna:     Parâmetro 'how' a ser passado para o método dropna, utilizado para limpar linhas/colunas com valores inexistentes
    :return:            Retorna a DF importada e tratada se a importação deu certo, caso contrário retorna uma DF vazia
    """
    
    try:
        """ ## Importando .csv """
        df = pd.read_csv(path, index_col=0)
        """ ## Excluindo colunas indesejadas """
        df.drop(dropcols, axis=1, inplace=True)
        """ ##  Limpando linhas/colunas com dados inexistentes """
        if clearna: 
            for i in range(2): df.dropna(how=clearna[i], axis=i, inplace=True)
    except (FileNotFoundError, AttributeError): return pd.DataFrame()
    else: return df

## Perguntas - Obesity among adults by country, 1975-2016

In [153]:
obesity_df = get_compact_df("..\\..\\Docs\\projetos\\obesity_&_gdp_analisys\\obesity_cleaned.csv")
display(obesity_df)

,Country,Year,Obesity (%),Sex
0,Afghanistan,1975,0.5 [0.2-1.1],Both sexes
1,Afghanistan,1975,0.2 [0.0-0.6],Male
2,Afghanistan,1975,0.8 [0.2-2.0],Female
3,Afghanistan,1976,0.5 [0.2-1.1],Both sexes
4,Afghanistan,1976,0.2 [0.0-0.7],Male
...,...,...,...,...
24565,Zimbabwe,2015,4.5 [2.4-7.6],Male
24566,Zimbabwe,2015,24.8 [18.9-31.3],Female
24567,Zimbabwe,2016,15.5 [12.0-19.2],Both sexes
24568,Zimbabwe,2016,4.7 [2.5-8.0],Male


##### Limpe os dados do DataFrame, criando uma coluna de nome 'Obesity' que conterá os valores de obesidade. Transforme em float as colunas que porventura foram importadas como string.

In [154]:
""" ## Dispondo informações sobre o DF importado """
# display(obesity_df.value_counts())
# display(obesity_df.info())

""" ## Criação da coluna 'Obesity'  """
obesity_df['Obesity'] = obesity_df['Obesity (%)'].apply(lambda x: x.split()[0])

""" ## Conversão de tipo colunas para numérico """
obesity_df['Obesity'] = pd.to_numeric(obesity_df['Obesity'], errors='coerce')
obesity_df.dropna(how='any', inplace=True)

display(obesity_df)

,Country,Year,Obesity (%),Sex,Obesity
0,Afghanistan,1975,0.5 [0.2-1.1],Both sexes,0.5
1,Afghanistan,1975,0.2 [0.0-0.6],Male,0.2
2,Afghanistan,1975,0.8 [0.2-2.0],Female,0.8
3,Afghanistan,1976,0.5 [0.2-1.1],Both sexes,0.5
4,Afghanistan,1976,0.2 [0.0-0.7],Male,0.2
...,...,...,...,...,...
24565,Zimbabwe,2015,4.5 [2.4-7.6],Male,4.5
24566,Zimbabwe,2015,24.8 [18.9-31.3],Female,24.8
24567,Zimbabwe,2016,15.5 [12.0-19.2],Both sexes,15.5
24568,Zimbabwe,2016,4.7 [2.5-8.0],Male,4.7


##### Qual o percentual médio de obesidade por sexo no mundo no ano de 2015?

In [155]:
obesity_2015_df = obesity_df[obesity_df['Year'] == 2015]
# display(obesity_2015_df)
obesity_2015_df['Obesity'].mean()

19.46282722513089

##### Quais são os 5 países com a maior e a menor taxa de aumento nos índices de obesidade no período observado?

In [163]:
obesity_2015_df['Obesity Increase Rate'] = 100 * (obesity_2015_df['Obesity'] / obesity_2015_df['Obesity'].shift())

min_inc_rate = obesity_2015_df.groupby('Country') [['Obesity Increase Rate']].min().sort_values('Obesity Increase Rate')
max_inc_rate = obesity_2015_df.groupby('Country') [['Obesity Increase Rate']].max().sort_values('Obesity Increase Rate')

display(min_inc_rate.head())
display(max_inc_rate.tail())

C:\Users\Administrativo\AppData\Local\Temp\ipykernel_14400\3776641453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obesity_2015_df['Obesity Increase Rate'] = (obesity_2015_df['Obesity'] / obesity_2015_df['Obesity'].shift()) * 100


,Obesity Increase Rate
Country,
Nepal,6.180666
Viet Nam,7.067138
Uganda,8.992806
Bangladesh,9.340659
Republic of Korea,10.563380


,Obesity Increase Rate
Country,
Palau,507.407407
Zimbabwe,551.111111
Lesotho,590.909091
Yemen,687.500000
Jordan,969.444444


##### Quais os países com maiores e menores níveis percetuais de obesidade em 2015?

In [173]:
min_obe_rate = obesity_2015_df.groupby('Country')[['Obesity']].min().sort_values('Obesity')
max_obe_rate = obesity_2015_df.groupby('Country')[['Obesity']].max().sort_values('Obesity')

display(min_obe_rate.head())
display(max_obe_rate.tail())


,Obesity
Country,
Viet Nam,1.5
Uganda,1.7
Ethiopia,1.8
Rwanda,1.8
Eritrea,1.9


,Obesity
Country,
Tuvalu,55.6
Marshall Islands,56.9
Palau,58.4
Cook Islands,58.8
Nauru,63.1


##### Qual a diferença média percentual de obesidade entre sexos ao longo dos anos para o Brasil?

In [172]:
obesity_2015_df

False

##### Você conseguiria plotar um gráfico mostrando a evolução da obesidade para ambos sexos no mundo?

In [ ]:
df_both.plot()

## Perguntas - GDP Per Person (1901-2011)

In [ ]:
df_gdp = pd.read_csv("Datasets/GDP.csv", decimal=".")

- Limpe o conjunto de dados, convertendo strings em datas ou float, quando necessário.

- Você conseguiria informar o primeiro valor registrado de cada país?

- Informe as regiões com maiores crescimentos de PIB per capita no século passado.

- Preecha os anos ausentes em cada país com uma estimativa, baseada na diferença entre o próximo registro e o anterior.

### DESAFIOS
- Você conseguiria criar um mapa do gdp ou da obesidade no mundo ao longo dos anos?

- Há uma relação entre níveis de obesidade e gdp per capita?